In [1]:
from influxdb.commands import InfluxDBQueries, DataProcessor
import warnings
import pandas as pd
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2024-01-01T01:00:00Z",stop="2025-01-01T01:00:00Z")

In [3]:
df_val = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-01-01T01:00:00Z", stop="2025-03-01T01:00:00Z")

In [4]:
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-03-01T01:00:00Z", stop="2025-04-01T01:00:00Z")

In [5]:
df_combined = pd.concat([df_train, df_test, df_val])

In [6]:
df_combined, scaler, columns = DataProcessor.add_features(df_combined)

Number of NaN values in each column:
high                0
low                 0
close               0
volume              0
open                0
return_1            1
return_5            5
rolling_mean_12    11
rolling_std_12     11
price_range         0
typical_price       0
vwap_20            19
vol_mean_10         9
vol_std_10          9
ma_7                6
ma_30              29
rsi_14             14
bb_mean_20         19
bb_std_20          19
bb_hband           19
bb_lband           19
macd                0
macd_signal         0
atr                23
obv                 0
dtype: int64
DatetimeIndex(['2024-01-01 01:15:00+00:00', '2024-01-01 01:20:00+00:00',
               '2024-01-01 01:25:00+00:00', '2024-01-01 01:30:00+00:00',
               '2024-01-01 01:35:00+00:00', '2024-01-01 01:40:00+00:00',
               '2024-01-01 01:45:00+00:00', '2024-01-01 01:50:00+00:00',
               '2024-01-01 01:55:00+00:00', '2024-01-01 02:00:00+00:00',
               ...
               '

In [7]:
df_train = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]
df_val = df_combined.loc[(df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')) &
                         (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ'))]
df_test = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]

In [8]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

In [9]:
X_train, y_train = DataProcessor.make_sequences(df_train, columns_to_select=columns, window_size=288, forecast_horizon=12)

In [10]:
X_val, y_val = DataProcessor.make_sequences(df_val, columns_to_select=columns, window_size=288, forecast_horizon=12)

In [11]:
from influxdb.commands import LSTMModel

Based on the previous cells in the notebook, I can see that `X_train.shape` would reveal the correct shape for the input. Looking at how the data is processed, I need to match the shape with the columns used in the LSTM model.



In [12]:
lstm_forecaster = LSTMModel(input_shape=(288, len(columns)), output_steps=12, lstm_units=16)

In [ ]:
lstm_forecaster.fit(X_train, y_train, X_val, y_val, 200, batch_size=128, patience=10)

Training started with EarlyStopping (patience=10)...
Epoch 1/200
818/818 ━━━━━━━━━━━━━━━━━━━━ 135s 160ms/step - directional_accuracy: 0.4990 - loss: 0.1131 - mse: 0.0524 - val_directional_accuracy: 0.5001 - val_loss: 0.0026 - val_mse: 1.8043e-05
Epoch 2/200
818/818 ━━━━━━━━━━━━━━━━━━━━ 133s 163ms/step - directional_accuracy: 0.5009 - loss: 0.0027 - mse: 2.1436e-05 - val_directional_accuracy: 0.4996 - val_loss: 0.0026 - val_mse: 1.8099e-05
Epoch 3/200
818/818 ━━━━━━━━━━━━━━━━━━━━ 139s 170ms/step - directional_accuracy: 0.4999 - loss: 0.0026 - mse: 1.7911e-05 - val_directional_accuracy: 0.4996 - val_loss: 0.0026 - val_mse: 1.7842e-05
Epoch 4/200
496/818 ━━━━━━━━━━━━━━━━━━━━ 53s 167ms/step - directional_accuracy: 0.5002 - loss: 0.0026 - mse: 1.7750e-05

In [ ]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=288, forecast_horizon=12)

In [ ]:
lstm_forecaster.model.evaluate(X_test, y_test)

In [ ]:
df_test_pred = df_test

In [ ]:
df_test_pred = df_test.iloc[:12]

In [ ]:
lstm_forecaster.model.save('model.h5')

In [ ]:
df_test_pred['close'] = pred[0]

In [ ]:
df_test_pred[columns] = scaler.inverse_transform(df_test_pred[columns])

In [ ]:
df_test_pred

In [ ]:
df_test[columns] = scaler.inverse_transform(df_test[columns])

In [ ]:
df_test

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df_test.iloc[:12]['close'], label='Actual')
plt.plot(df_test_pred['close'], label='Predicted')
plt.legend()